In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import duckdb
load_dotenv()

False

In [2]:
csv_file = r"C:\Users\USER\Desktop\final_datasets\merged_sales_csv\raw_product_sales_2019.csv"
states = r"C:\Users\USER\Desktop\elt_test_run\md_product_sales\csv_files\states.csv"
df = pd.read_csv(csv_file)
states_df = pd.read_csv(states)
con = duckdb.connect()

In [3]:
t_cast_types = con.sql(
    """
    with t_rm_nulls_dups as (
        select distinct *
        from df
        where "Order ID" != 'Order ID' and "Order ID" is not null
    ),
    t_cast_types as (
    select
        "Order ID"::int as order_id,
        "Product"::varchar(255) as product,
        "Quantity Ordered"::int as qty_ordered,
        "Price Each"::numeric(10,2) as price_each,
        case
            when
                length("Order Date") = 14 then strptime("Order Date", '%m/%d/%y %H:%M')
            else strptime("Order Date", '%m/%d/%Y %H:%M')
        end as order_date,
        "Purchase Address"::varchar(255) as purchase_address,
        "Store"::varchar(255) as store,
        "Payment Method"::varchar(255) as payment_method
    from t_rm_nulls_dups
    )
    select *
    from t_cast_types
    """
    ).df()

In [4]:
t_cast_types.head()

,order_id,product,qty_ordered,price_each,order_date,purchase_address,store,payment_method
0,220211,Wired Headphones,1,11.99,2019-06-14 23:15:00,"914 12th St, Portland, OR 97035",Walmart,Credit
1,220216,Wired Headphones,2,11.99,2019-06-07 11:40:00,"989 7th St, San Francisco, CA 94016",Walmart,Credit
2,220221,Bose SoundSport Headphones,1,99.99,2019-06-01 12:22:00,"843 12th St, Seattle, WA 98101",eBay,Credit
3,220223,AAA Batteries (4-pack),3,2.99,2019-06-19 15:20:00,"995 Spruce St, Austin, TX 73301",Amazon,Credit
4,220269,Bose SoundSport Headphones,1,99.99,2019-06-13 20:41:00,"457 Spruce St, San Francisco, CA 94016",Amazon,Credit


In [5]:
final_product_sales = """ 
with pre_final_product_sales as (
	select
		row_number() over(order by order_date asc) as order_sk_id,
		order_id,
		product,
		price_each,
		qty_ordered,
		order_date,
		purchase_address,
		store,
		payment_method
	from t_cast_types
	where extract(year from order_date) = 2019
	order by order_date asc
)

select *
from pre_final_product_sales

"""
final_product_sales = duckdb.sql(final_product_sales).df()
# print(q_final_product_sales)

In [6]:
final_product_sales.head()

,order_sk_id,order_id,product,price_each,qty_ordered,order_date,purchase_address,store,payment_method
0,1,147268,Wired Headphones,11.99,1,2019-01-01 03:07:00,"9 Lake St, New York City, NY 10001",Walmart,Credit
1,2,148041,USB-C Charging Cable,11.95,1,2019-01-01 03:40:00,"760 Church St, San Francisco, CA 94016",eBay,Credit
2,3,149343,Apple Airpods Headphones,150.00,1,2019-01-01 04:56:00,"735 5th St, New York City, NY 10001",eBay,Credit
3,4,149964,AAA Batteries (4-pack),2.99,1,2019-01-01 05:53:00,"75 Jackson St, Dallas, TX 75001",eBay,Credit
4,5,149350,USB-C Charging Cable,11.95,2,2019-01-01 06:03:00,"943 2nd St, Atlanta, GA 30301",Target,Cash


In [7]:
dim_order_dates = """
with dim_order_dates as (
	select
		row_number() over(order by order_date asc) as orderdate_sk_id,
		order_date as order_time_stamp,
  		cast((year(order_date)||'-'||month(order_date)||'-'||day(order_date)) as DATE) as date,
		extract(year from order_date) as year,
		extract(month from order_date) as month,
		monthname(order_date) as month_name,
		dayname(order_date) as day_name,
		extract(day from order_date) as day_month,
		extract(dow from order_date) as day_name_num,
		extract(hour from order_date) as hour,
		extract(minute from order_date) as minute
	from (select
		distinct order_date
	from final_product_sales) distinct_order_date
)

select *
from dim_order_dates
"""
dim_order_dates = duckdb.sql(dim_order_dates).df()

In [8]:
dim_order_dates.head()

,orderdate_sk_id,order_time_stamp,date,year,month,month_name,day_name,day_month,day_name_num,hour,minute
0,1,2019-01-01 03:07:00,2019-01-01,2019,1,January,Tuesday,1,2,3,7
1,2,2019-01-01 03:40:00,2019-01-01,2019,1,January,Tuesday,1,2,3,40
2,3,2019-01-01 04:56:00,2019-01-01,2019,1,January,Tuesday,1,2,4,56
3,4,2019-01-01 05:53:00,2019-01-01,2019,1,January,Tuesday,1,2,5,53
4,5,2019-01-01 06:03:00,2019-01-01,2019,1,January,Tuesday,1,2,6,3


In [9]:
dim_stores = con.sql(
    """
    with dim_stores as (
    select
        row_number() over(order by lower(store) asc) as store_sk_id,
        row_number() over(order by lower(store) asc) as store_id,
        store
    from (
        select distinct store
        from final_product_sales ) as distinct_store 
    )
    select distinct *
    from dim_stores
    order by store_id asc
    """
).df()

In [10]:
dim_stores.head()

,store_sk_id,store_id,store
0,1,1,Amazon
1,2,2,eBay
2,3,3,Target
3,4,4,Walmart


In [11]:
dim_payment_methods = con.sql(
    """
    with pre_dim_payment_methods as (
    select
        row_number() over(order by payment_method asc) as pay_method_sk_id,
        row_number() over(order by payment_method asc) as pay_method_id,
        payment_method
    from (
    select distinct payment_method
    from final_product_sales
    order by payment_method asc ) distinct_pay_methods   
    )

    select *
    from pre_dim_payment_methods
    """
).df()

In [12]:
dim_payment_methods.head()

,pay_method_sk_id,pay_method_id,payment_method
0,1,1,Cash
1,2,2,Credit


In [13]:
dim_products = con.sql(
    """
    with dim_products as (
        select
            row_number() over(order by lower(product) asc) as product_sk_id,
            row_number() over(order by lower(product) asc) as product_id,
            product,
            price_each
        from (
            select distinct product, price_each
            from final_product_sales
            order by lower(product) asc
        ) distinct_products
    )
    select distinct *
    from dim_products
    order by product_id asc
    """
).df()

In [14]:
dim_products.head()

,product_sk_id,product_id,product,price_each
0,1,1,20in Monitor,109.99
1,2,2,27in 4K Gaming Monitor,389.99
2,3,3,27in FHD Monitor,149.99
3,4,4,34in Ultrawide Monitor,379.99
4,5,5,AA Batteries (4-pack),3.84


In [15]:
dim_states = con.sql(
    """
    select *
    from states_df
    """
).df()

In [16]:
dim_states.head()

,state_id,state_name,state_abbrv,capital,latitude,longitude
0,1,Alabama,AL,Montgomery,32.377716,-86.300568
1,2,Alaska,AK,Juneau,58.301598,-134.420212
2,3,American Samoa,AS,Pago Pago,-14.279444,-170.700556
3,4,Arizona,AZ,Phoenix,33.448143,-112.096962
4,5,Arkansas,AR,Little Rock,34.746613,-92.288986


In [17]:
dim_customers = con.sql(
    """
    with stg_dim_customers as (
	select
		row_number() over(order by purchase_address asc) customer_sk_id,
		row_number() over(order by purchase_address asc) customer_id,
		purchase_address,
		ltrim(split_part(purchase_address,',',2))::varchar as city,
		split_part(ltrim(split_part(purchase_address,',',3)),' ',1)::varchar as cust_state_abbrv
	from ( select distinct purchase_address
	from final_product_sales ) distinct_address
    ),
	    dim_customers as (
            select
            a.customer_sk_id,
            a.customer_id,
            a.purchase_address,
            a.city,
            a.cust_state_abbrv,
            b.state_name,
            b.capital,
            b.latitude,
            b.longitude
            from stg_dim_customers as a
		left join dim_states as b on a.cust_state_abbrv = b.state_abbrv
    )

    select *
    from dim_customers
    """
).df()

In [18]:
dim_customers.head()

,customer_sk_id,customer_id,purchase_address,city,cust_state_abbrv,state_name,capital,latitude,longitude
0,124531,124531,"895 West St, Seattle, WA 98101",Seattle,WA,Washington,Olympia,47.035805,-122.905014
1,124532,124532,"895 Willow St, Austin, TX 73301",Austin,TX,Texas,Austin,30.274670,-97.740349
2,124533,124533,"895 Willow St, New York City, NY 10001",New York City,NY,New York,Albany,35.780430,-78.639099
3,124534,124534,"895 Wilson St, Boston, MA 02215",Boston,MA,Massachusetts,Boston,42.358162,-71.063698
4,124535,124535,"895 Wilson St, Los Angeles, CA 90001",Los Angeles,CA,California,Sacramento,38.576668,-121.493629


In [19]:
final_fact_sales = con.sql(
    """
	select
	a.order_sk_id,
	a.order_id,
	c.customer_sk_id,
	b.product_sk_id,
	a.qty_ordered,
	(a.qty_ordered * b.price_each)::numeric(10,2) as total_price,
	c.customer_sk_id as purchase_address_id,
	f.orderdate_sk_id,
	d.store_sk_id,
	e.pay_method_sk_id
	
	from final_product_sales as a
	left join dim_products as b on a.product = b.product
	left join dim_customers as c on a.purchase_address = c.purchase_address
	left join dim_stores as d on a.store = d.store
	left join dim_payment_methods as e on a.payment_method = e.payment_method
	left join dim_order_dates as f on a.order_date = f.order_time_stamp
	order by order_sk_id asc
    """
).df()

In [20]:
final_fact_sales.head()

,order_sk_id,order_id,customer_sk_id,product_sk_id,qty_ordered,total_price,purchase_address_id,orderdate_sk_id,store_sk_id,pay_method_sk_id
0,1,147268,125160,19,1,11.99,125160,1,4,2
1,2,148041,103419,17,1,11.95,103419,2,2,2
2,3,149343,99418,7,1,150.00,99418,3,2,2
3,4,149964,101715,6,1,2.99,101715,4,2,2
4,5,149350,132005,17,2,23.90,132005,5,3,1


In [21]:
mart_product_sales = con.sql(
    """
    select
	a.order_id,
	c.customer_id,
	b.product,
	b.price_each,
	a.qty_ordered,
	a.total_price,
	f.order_time_stamp,
	f.date,
	f.month,
	f.month_name,
	f.day_name,
	f.day_name_num,
	f.day_month,
	c.purchase_address,
	c.city,
	c.state_name,
    c.capital,
    c.latitude,
    c.longitude,
	d.store,
	e.payment_method
	from final_fact_sales as a
	left join dim_products as b on a.product_sk_id = b.product_sk_id
	left join dim_customers as c on a.customer_sk_id = c.customer_sk_id
	left join dim_stores as d on a.store_sk_id = d.store_sk_id
	left join dim_payment_methods as e on a.pay_method_sk_id = e.pay_method_sk_id
	left join dim_order_dates as f on a.orderdate_sk_id = f.orderdate_sk_id
    order by order_time_stamp asc
    """
).df()

In [22]:
mart_product_sales.head()

,order_id,customer_id,product,price_each,qty_ordered,total_price,order_time_stamp,date,month,month_name,...,day_name_num,day_month,purchase_address,city,state_name,capital,latitude,longitude,store,payment_method
0,147268,125160,Wired Headphones,11.99,1,11.99,2019-01-01 03:07:00,2019-01-01,1,January,...,2,1,"9 Lake St, New York City, NY 10001",New York City,New York,Albany,35.780430,-78.639099,Walmart,Credit
1,148041,103419,USB-C Charging Cable,11.95,1,11.95,2019-01-01 03:40:00,2019-01-01,1,January,...,2,1,"760 Church St, San Francisco, CA 94016",San Francisco,California,Sacramento,38.576668,-121.493629,eBay,Credit
2,149343,99418,Apple Airpods Headphones,150.00,1,150.00,2019-01-01 04:56:00,2019-01-01,1,January,...,2,1,"735 5th St, New York City, NY 10001",New York City,New York,Albany,35.780430,-78.639099,eBay,Credit
3,149964,101715,AAA Batteries (4-pack),2.99,1,2.99,2019-01-01 05:53:00,2019-01-01,1,January,...,2,1,"75 Jackson St, Dallas, TX 75001",Dallas,Texas,Austin,30.274670,-97.740349,eBay,Credit
4,149350,132005,USB-C Charging Cable,11.95,2,23.90,2019-01-01 06:03:00,2019-01-01,1,January,...,2,1,"943 2nd St, Atlanta, GA 30301",Atlanta,Georgia,Atlanta,30.438118,-84.281296,Target,Cash


In [23]:
mart_product_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185652 entries, 0 to 185651
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   order_id          185652 non-null  int32         
 1   customer_id       185652 non-null  int64         
 2   product           185652 non-null  object        
 3   price_each        185652 non-null  float64       
 4   qty_ordered       185652 non-null  int32         
 5   total_price       185652 non-null  float64       
 6   order_time_stamp  185652 non-null  datetime64[ns]
 7   date              185652 non-null  datetime64[ns]
 8   month             185652 non-null  int64         
 9   month_name        185652 non-null  object        
 10  day_name          185652 non-null  object        
 11  day_name_num      185652 non-null  int64         
 12  day_month         185652 non-null  int64         
 13  purchase_address  185652 non-null  object        
 14  city